In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from tqdm import trange
import random
import math
from scipy import interp
import statistics 
import os

from tcrmodels.ergo2.model import ERGO2
from tcrmodels.nettcr2.model import NetTCR2
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve, roc_curve, auc

from matplotlib import collections
from matplotlib import colors
from numpy.random import normal

Using TensorFlow backend.


In [2]:
metrics = [
    'AUROC',
    'Accuracy',
    'Recall',
    'Precision',
    'F1 score',
    'AUPR'
]

def pr_auc(y_true, y_prob):
    precision, recall, thresholds = precision_recall_curve(y_true, y_prob)
    pr_auc = auc(recall, precision)
    return pr_auc

def get_scores(y_true, y_prob, y_pred):
    """
    Compute a df with all classification metrics and respective scores.
    """
    
    scores = [
        roc_auc_score(y_true, y_prob),
        accuracy_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        f1_score(y_true, y_pred),
        pr_auc(y_true, y_prob)
    ]
    
    df = pd.DataFrame(data={'score': scores, 'metrics': metrics})
    return df

In [3]:
def set_random_seed(random_seed):
    random.seed(random_seed)
    np.random.seed(random_seed)

In [5]:
login = os.getlogin( )
DATA_BASE = f"/home/{login}/Git/tc-hard/notebooks/notebooks.classification.vdjdb-high-scores/results/"
FIGURE_BASE = f"/home/{login}/Git/tc-hard/notebooks/notebooks.classification-results.vdjdb-high-scores/figures/"

# NetTCR-2.0 - Hard split (Test: only randomized negatives) - Train: only randomized negatives

In [7]:
results_nettcr2 = []

for i in tqdm(range(5)):
    df_train = pd.read_csv(DATA_BASE+"train/only-sampled-negs/"+f"train-{i}.csv")
    df_test = pd.read_csv(DATA_BASE+"test/only-sampled-negs/"+f"test-{i}.csv")
    
    max_pep_len = max(df_train["antigen.epitope"].str.len().max(), df_test["antigen.epitope"].str.len().max())
    max_cdr3b_len = max(df_train["cdr3.beta"].str.len().max(), df_test["cdr3.beta"].str.len().max())


    model = NetTCR2(
        architecture="b", 
        single_chain_column='cdr3.beta',
        peptide_column='antigen.epitope',
        label_column='label',
        max_pep_len=max_pep_len, 
        max_cdr3_len=max_cdr3b_len
    )
    model.train(df_train, epochs=1000);

    prediction_df = model.test(df_test)

    scores_df = get_scores(
        y_true=prediction_df['label'].to_numpy(), 
        y_prob=prediction_df['prediction'].to_numpy(),
        y_pred=prediction_df['prediction'].to_numpy().round(),
    )
    scores_df['experiment'] = i
    results_nettcr2.append(scores_df)
    df_test['prediction'] = prediction_df['prediction']
    df_test.to_csv(RESULTS_BASE+f"nettcr2.pep+cdr3b.only-sampled-negs.hard-split.{i}.csv", index=False)

results_nettcr2_df = pd.concat(results_nettcr2)
results_nettcr2_df.to_csv(RESULTS_BASE+"nettcr2.pep+cdr3b.only-sampled-negs.hard-split.csv", index=False)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/1000
8585/8585 [==============================] - 1s 105us/step - loss: 0.6510
Epoch 2/1000
8585/8585 [==============================] - 0s 36us/step - loss: 0.6438
Epoch 3/1000
8585/8585 [==============================] - 0s 34us/step - loss: 0.6434
Epoch 4/1000
8585/8585 [==============================] - 0s 34us/step - loss: 0.6429
Epoch 5/1000
8585/8585 [==============================] - 0s 34us/step - loss: 0.6434
Epoch 6/1000
8585/8585 [==============================] - 0s 33us/step - loss: 0.6437
Epoch 7/1000
8585/8585 [==============================] - 0s 34us/step - loss: 0.6436
Epoch 8/1000
8585/8585 [==============================] - 0s 35us/step - loss: 0.6427
Epoch 9/1000
8585/8585 [==============================] - 0s 35us/step - loss: 0.6427
Epoch 10/1000
8585/8585 [==============================] - 0s 35us/step - loss: 0.6423
Epoch 11/1000
8585/8585 [==============================] - 0s 35us/step - loss: 0.6424
Epoch 12/1000
8585/8585 [==========================

8585/8585 [==============================] - 0s 35us/step - loss: 0.3063
Epoch 189/1000
8585/8585 [==============================] - 0s 35us/step - loss: 0.3039
Epoch 190/1000
8585/8585 [==============================] - 0s 35us/step - loss: 0.3023
Epoch 191/1000
8585/8585 [==============================] - 0s 35us/step - loss: 0.2993
Epoch 192/1000
8585/8585 [==============================] - 0s 35us/step - loss: 0.2990
Epoch 193/1000
8585/8585 [==============================] - 0s 35us/step - loss: 0.2965
Epoch 194/1000
8585/8585 [==============================] - 0s 35us/step - loss: 0.3014
Epoch 195/1000
8585/8585 [==============================] - 0s 35us/step - loss: 0.2997
Epoch 196/1000
8585/8585 [==============================] - 0s 35us/step - loss: 0.2965
Epoch 197/1000
8585/8585 [==============================] - 0s 34us/step - loss: 0.2991
Epoch 198/1000
8585/8585 [==============================] - 0s 35us/step - loss: 0.2926
Epoch 199/1000
8585/8585 [=====================

8585/8585 [==============================] - 0s 33us/step - loss: 0.1290
Epoch 375/1000
8585/8585 [==============================] - 0s 34us/step - loss: 0.1333
Epoch 376/1000
8585/8585 [==============================] - 0s 34us/step - loss: 0.1313
Epoch 377/1000
8585/8585 [==============================] - 0s 34us/step - loss: 0.1257
Epoch 378/1000
8585/8585 [==============================] - 0s 35us/step - loss: 0.1267
Epoch 379/1000
8585/8585 [==============================] - 0s 35us/step - loss: 0.1327
Epoch 380/1000
8585/8585 [==============================] - 0s 35us/step - loss: 0.1256
Epoch 381/1000
8585/8585 [==============================] - 0s 35us/step - loss: 0.1285
Epoch 382/1000
8585/8585 [==============================] - 0s 35us/step - loss: 0.1245
Epoch 383/1000
8585/8585 [==============================] - 0s 35us/step - loss: 0.1219
Epoch 384/1000
8585/8585 [==============================] - 0s 34us/step - loss: 0.1269
Epoch 385/1000
8585/8585 [=====================

8585/8585 [==============================] - 0s 34us/step - loss: 0.0362
Epoch 561/1000
8585/8585 [==============================] - 0s 33us/step - loss: 0.0359
Epoch 562/1000
8585/8585 [==============================] - 0s 35us/step - loss: 0.0388
Epoch 563/1000
8585/8585 [==============================] - 0s 33us/step - loss: 0.0351
Epoch 564/1000
8585/8585 [==============================] - 0s 33us/step - loss: 0.0349
Epoch 565/1000
8585/8585 [==============================] - 0s 33us/step - loss: 0.0396
Epoch 566/1000
8585/8585 [==============================] - 0s 34us/step - loss: 0.0336
Epoch 567/1000
8585/8585 [==============================] - 0s 34us/step - loss: 0.0310
Epoch 568/1000
8585/8585 [==============================] - 0s 34us/step - loss: 0.0311
Epoch 569/1000
8585/8585 [==============================] - 0s 33us/step - loss: 0.0319
Epoch 570/1000
8585/8585 [==============================] - 0s 33us/step - loss: 0.0325
Epoch 571/1000
8585/8585 [=====================

...

8598/8598 [==============================] - 0s 37us/step - loss: 0.0302
Epoch 576/1000
8598/8598 [==============================] - 0s 37us/step - loss: 0.0293
Epoch 577/1000
8598/8598 [==============================] - 0s 37us/step - loss: 0.0317
Epoch 578/1000
8598/8598 [==============================] - 0s 37us/step - loss: 0.0299
Epoch 579/1000
8598/8598 [==============================] - 0s 37us/step - loss: 0.0292
Epoch 580/1000
8598/8598 [==============================] - 0s 37us/step - loss: 0.0309
Epoch 581/1000
8598/8598 [==============================] - 0s 36us/step - loss: 0.0289
Epoch 582/1000
8598/8598 [==============================] - 0s 37us/step - loss: 0.0284
Epoch 583/1000
8598/8598 [==============================] - 0s 37us/step - loss: 0.0283
Epoch 584/1000
8598/8598 [==============================] - 0s 37us/step - loss: 0.0268
Epoch 585/1000
8598/8598 [==============================] - 0s 42us/step - loss: 0.0278
Epoch 586/1000
8598/8598 [=====================

 80%|████████  | 4/5 [12:16<03:10, 190.53s/it]

Epoch 1/1000
8613/8613 [==============================] - 1s 156us/step - loss: 0.6564
Epoch 2/1000
8613/8613 [==============================] - 0s 45us/step - loss: 0.6444
Epoch 3/1000
8613/8613 [==============================] - 0s 42us/step - loss: 0.6437
Epoch 4/1000
8613/8613 [==============================] - 0s 36us/step - loss: 0.6432
Epoch 5/1000
8613/8613 [==============================] - 0s 39us/step - loss: 0.6436
Epoch 6/1000
8613/8613 [==============================] - 0s 36us/step - loss: 0.6438
Epoch 7/1000
8613/8613 [==============================] - 0s 36us/step - loss: 0.6444
Epoch 8/1000
8613/8613 [==============================] - 0s 36us/step - loss: 0.6427
Epoch 9/1000
8613/8613 [==============================] - 0s 36us/step - loss: 0.6428
Epoch 10/1000
8613/8613 [==============================] - 0s 36us/step - loss: 0.6429
Epoch 11/1000
8613/8613 [==============================] - 0s 36us/step - loss: 0.6443
Epoch 12/1000
8613/8613 [==========================

8613/8613 [==============================] - 0s 35us/step - loss: 0.2955
Epoch 189/1000
8613/8613 [==============================] - 0s 35us/step - loss: 0.2940
Epoch 190/1000
8613/8613 [==============================] - 0s 36us/step - loss: 0.2923
Epoch 191/1000
8613/8613 [==============================] - 0s 38us/step - loss: 0.2896
Epoch 192/1000
8613/8613 [==============================] - 0s 39us/step - loss: 0.2897
Epoch 193/1000
8613/8613 [==============================] - 0s 35us/step - loss: 0.2910
Epoch 194/1000
8613/8613 [==============================] - 0s 36us/step - loss: 0.2841
Epoch 195/1000
8613/8613 [==============================] - 0s 36us/step - loss: 0.2883
Epoch 196/1000
8613/8613 [==============================] - 0s 36us/step - loss: 0.2840
Epoch 197/1000
8613/8613 [==============================] - 0s 36us/step - loss: 0.2840
Epoch 198/1000
8613/8613 [==============================] - 0s 35us/step - loss: 0.2795
Epoch 199/1000
8613/8613 [=====================

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# NetTCR-2.0 - Random split - Train and test: only randomized negatives

In [10]:
results_nettcr2 = []

for i in tqdm(range(5)):
    df = pd.read_csv(f"/mnt/container-nle-tcr/tc-hard-data/tc-hard/ds.vdjdb-high-scores.csv")

    df = df.drop_duplicates(
        subset=["antigen.epitope", "cdr3.beta", "label"], keep="first"
    ).reset_index(drop=True)
    
    df_train, df_test = train_test_split(df, test_size=0.2, random_state=i)
    
    max_pep_len = max(df_train["antigen.epitope"].str.len().max(), df_test["antigen.epitope"].str.len().max())
    max_cdr3b_len = max(df_train["cdr3.beta"].str.len().max(), df_test["cdr3.beta"].str.len().max())


    model = NetTCR2(
        architecture="b", 
        single_chain_column='cdr3.beta',
        peptide_column='antigen.epitope',
        label_column='label',
        max_pep_len=max_pep_len, 
        max_cdr3_len=max_cdr3b_len
    )
    model.train(df_train, epochs=1000);

    prediction_df = model.test(df_test)

    scores_df = get_scores(
        y_true=prediction_df['label'].to_numpy(), 
        y_prob=prediction_df['prediction'].to_numpy(),
        y_pred=prediction_df['prediction'].to_numpy().round(),
    )
    scores_df['experiment'] = i
    results_nettcr2.append(scores_df)
    df_test['prediction'] = prediction_df['prediction']
    df_test.to_csv(RESULTS_BASE+f"nettcr2.pep+cdr3b.only-sampled-negs.random-split.{i}.csv", index=False)
    
results_nettcr2_df = pd.concat(results_nettcr2)
results_nettcr2_df.to_csv(RESULTS_BASE+"nettcr2.pep+cdr3b.only-sampled-negs.random-split.csv", index=False)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/1000
8107/8107 [==============================] - 1s 175us/step - loss: 0.6778
Epoch 2/1000
8107/8107 [==============================] - 0s 40us/step - loss: 0.6449
Epoch 3/1000
8107/8107 [==============================] - 0s 39us/step - loss: 0.6454
Epoch 4/1000
8107/8107 [==============================] - 0s 37us/step - loss: 0.6453
Epoch 5/1000
8107/8107 [==============================] - 0s 42us/step - loss: 0.6448
Epoch 6/1000
8107/8107 [==============================] - 0s 43us/step - loss: 0.6453
Epoch 7/1000
8107/8107 [==============================] - 0s 38us/step - loss: 0.6451
Epoch 8/1000
8107/8107 [==============================] - 0s 39us/step - loss: 0.6451
Epoch 9/1000
8107/8107 [==============================] - 0s 38us/step - loss: 0.6450
Epoch 10/1000
8107/8107 [==============================] - 0s 39us/step - loss: 0.6445
Epoch 11/1000
8107/8107 [==============================] - 0s 46us/step - loss: 0.6440
Epoch 12/1000
8107/8107 [==========================

8107/8107 [==============================] - 0s 38us/step - loss: 0.3750
Epoch 189/1000
8107/8107 [==============================] - 0s 38us/step - loss: 0.3746
Epoch 190/1000
8107/8107 [==============================] - 0s 38us/step - loss: 0.3723
Epoch 191/1000
8107/8107 [==============================] - 0s 38us/step - loss: 0.3714
Epoch 192/1000
8107/8107 [==============================] - 0s 38us/step - loss: 0.3738
Epoch 193/1000
8107/8107 [==============================] - 0s 39us/step - loss: 0.3705
Epoch 194/1000
8107/8107 [==============================] - 0s 41us/step - loss: 0.3707
Epoch 195/1000
8107/8107 [==============================] - 0s 38us/step - loss: 0.3649
Epoch 196/1000
8107/8107 [==============================] - 0s 38us/step - loss: 0.3659
Epoch 197/1000
8107/8107 [==============================] - 0s 38us/step - loss: 0.3689
Epoch 198/1000
8107/8107 [==============================] - 0s 38us/step - loss: 0.3640
Epoch 199/1000
8107/8107 [=====================

8107/8107 [==============================] - 0s 37us/step - loss: 0.1824
Epoch 375/1000
8107/8107 [==============================] - 0s 40us/step - loss: 0.1821
Epoch 376/1000
8107/8107 [==============================] - 0s 39us/step - loss: 0.1788
Epoch 377/1000
8107/8107 [==============================] - 0s 39us/step - loss: 0.1772
Epoch 378/1000
8107/8107 [==============================] - 0s 39us/step - loss: 0.1808
Epoch 379/1000
8107/8107 [==============================] - 0s 38us/step - loss: 0.1724
Epoch 380/1000
8107/8107 [==============================] - 0s 38us/step - loss: 0.1742
Epoch 381/1000
8107/8107 [==============================] - 0s 38us/step - loss: 0.1704
Epoch 382/1000
8107/8107 [==============================] - 0s 38us/step - loss: 0.1759
Epoch 383/1000
8107/8107 [==============================] - 0s 38us/step - loss: 0.1707
Epoch 384/1000
8107/8107 [==============================] - 0s 38us/step - loss: 0.1702
Epoch 385/1000
8107/8107 [=====================

8107/8107 [==============================] - 0s 37us/step - loss: 0.0617
Epoch 561/1000
8107/8107 [==============================] - 0s 37us/step - loss: 0.0597
Epoch 562/1000
8107/8107 [==============================] - 0s 37us/step - loss: 0.0571
Epoch 563/1000
8107/8107 [==============================] - 0s 38us/step - loss: 0.0568
Epoch 564/1000
8107/8107 [==============================] - 0s 38us/step - loss: 0.0578
Epoch 565/1000
8107/8107 [==============================] - 0s 37us/step - loss: 0.0572
Epoch 566/1000
8107/8107 [==============================] - 0s 37us/step - loss: 0.0548
Epoch 567/1000
8107/8107 [==============================] - 0s 37us/step - loss: 0.0542
Epoch 568/1000
8107/8107 [==============================] - 0s 37us/step - loss: 0.0564
Epoch 569/1000
8107/8107 [==============================] - 0s 40us/step - loss: 0.0558
Epoch 570/1000
8107/8107 [==============================] - 0s 38us/step - loss: 0.0537
Epoch 571/1000
8107/8107 [=====================

...

Epoch 1/1000
8107/8107 [==============================] - 2s 214us/step - loss: 0.6602
Epoch 2/1000
8107/8107 [==============================] - 0s 44us/step - loss: 0.6418
Epoch 3/1000
8107/8107 [==============================] - 0s 39us/step - loss: 0.6418
Epoch 4/1000
8107/8107 [==============================] - 0s 42us/step - loss: 0.6413
Epoch 5/1000
8107/8107 [==============================] - 0s 45us/step - loss: 0.6412
Epoch 6/1000
8107/8107 [==============================] - 0s 40us/step - loss: 0.6411
Epoch 7/1000
8107/8107 [==============================] - 0s 42us/step - loss: 0.6412
Epoch 8/1000
8107/8107 [==============================] - 0s 39us/step - loss: 0.6406
Epoch 9/1000
8107/8107 [==============================] - 0s 39us/step - loss: 0.6410
Epoch 10/1000
8107/8107 [==============================] - 0s 39us/step - loss: 0.6428
Epoch 11/1000
8107/8107 [==============================] - 0s 39us/step - loss: 0.6414
Epoch 12/1000
8107/8107 [==========================

8107/8107 [==============================] - 0s 42us/step - loss: 0.3710
Epoch 189/1000
8107/8107 [==============================] - 0s 42us/step - loss: 0.3697
Epoch 190/1000
8107/8107 [==============================] - 0s 42us/step - loss: 0.3679
Epoch 191/1000
8107/8107 [==============================] - 0s 43us/step - loss: 0.3652
Epoch 192/1000
8107/8107 [==============================] - 0s 44us/step - loss: 0.3672
Epoch 193/1000
8107/8107 [==============================] - 0s 41us/step - loss: 0.3673
Epoch 194/1000
8107/8107 [==============================] - 0s 41us/step - loss: 0.3633
Epoch 195/1000
8107/8107 [==============================] - 0s 41us/step - loss: 0.3635
Epoch 196/1000
8107/8107 [==============================] - 0s 41us/step - loss: 0.3630
Epoch 197/1000
8107/8107 [==============================] - 0s 41us/step - loss: 0.3640
Epoch 198/1000
8107/8107 [==============================] - 0s 41us/step - loss: 0.3583
Epoch 199/1000
8107/8107 [=====================

8107/8107 [==============================] - 0s 41us/step - loss: 0.1937
Epoch 375/1000
8107/8107 [==============================] - 0s 41us/step - loss: 0.1938
Epoch 376/1000
8107/8107 [==============================] - 0s 41us/step - loss: 0.1946
Epoch 377/1000
8107/8107 [==============================] - 0s 41us/step - loss: 0.1949
Epoch 378/1000
8107/8107 [==============================] - 0s 41us/step - loss: 0.1864
Epoch 379/1000
8107/8107 [==============================] - 0s 41us/step - loss: 0.1914
Epoch 380/1000
8107/8107 [==============================] - 0s 39us/step - loss: 0.1878
Epoch 381/1000
8107/8107 [==============================] - 0s 38us/step - loss: 0.1889
Epoch 382/1000
8107/8107 [==============================] - 0s 38us/step - loss: 0.1863
Epoch 383/1000
8107/8107 [==============================] - 0s 38us/step - loss: 0.1887
Epoch 384/1000
8107/8107 [==============================] - 0s 38us/step - loss: 0.1846
Epoch 385/1000
8107/8107 [=====================

8107/8107 [==============================] - 0s 39us/step - loss: 0.0845
Epoch 561/1000
8107/8107 [==============================] - 0s 39us/step - loss: 0.0797
Epoch 562/1000
8107/8107 [==============================] - 0s 39us/step - loss: 0.0797
Epoch 563/1000
8107/8107 [==============================] - 0s 39us/step - loss: 0.0817
Epoch 564/1000
8107/8107 [==============================] - 0s 39us/step - loss: 0.0870
Epoch 565/1000
8107/8107 [==============================] - 0s 39us/step - loss: 0.0795
Epoch 566/1000
8107/8107 [==============================] - 0s 39us/step - loss: 0.0791
Epoch 567/1000
8107/8107 [==============================] - 0s 39us/step - loss: 0.0778
Epoch 568/1000
8107/8107 [==============================] - 0s 39us/step - loss: 0.0808
Epoch 569/1000
8107/8107 [==============================] - 0s 38us/step - loss: 0.0809
Epoch 570/1000
8107/8107 [==============================] - 0s 39us/step - loss: 0.0783
Epoch 571/1000
8107/8107 [=====================

/home/nle-fgrazioli/anaconda3/envs/tcrmodels/lib/python3.6/site-packages/tcrmodels/nettcr2/model.py:199: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['prediction'] = preds
/home/nle-fgrazioli/anaconda3/envs/tcrmodels/lib/python3.6/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 5/5 [16:31<00:00, 198.36s/it]
